In [26]:
# Baisc Implementation 
import torch 
import torch.nn as nn  
import torch.optim as optim
import torchvision 
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [ ]:
class Discriminator(nn.Module):
  def __init__(self,img_dim):
    super(Discriminator,self).__init__()
    self.disc = nn.Sequential(
        nn.Linear(img_dim,128),
        nn.LeakyReLU(0.1),
        nn.Linear(128,1),
        nn.Sigmoid())
    
  def forward(self,x):
    return self.disc(x)


class Generator(nn.Module):
  def __init__(self, z_dim, img_dim):
    super(Generator,self).__init__()
    self.gen = nn.Sequential(
        nn.Linear(z_dim,256),
        nn.LeakyReLU(0.1),
        nn.Linear(256,img_dim),
        nn.Tanh())
    
  def forward(self,x):
    return self.gen(x)


# Gan Are sensitive to hyperparameters 
device = "cuda" if torch.cuda.is_available() else "cpu"
lr = 3e-4   
z_dim = 64
img_dim = 28*28*1
batch_size = 32
num_epoch = 50

disc = Discriminator(img_dim).to(device)
gen = Generator(z_dim,img_dim).to(device)

fixed_noise = torch.randn((batch_size, z_dim)).to(device)

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,),(0.5,))])
dataset = datasets.MNIST(root='dataset/', transform=transform, download=True)
loader = DataLoader(dataset,batch_size=batch_size, shuffle=True)

optimizer_disc = optim.Adam(disc.parameters(),lr=lr)
optimizer_gen = optim.Adam(gen.parameters(),lr=lr)

critrion = nn.BCELoss()

writer_fake = SummaryWriter(f'GAN_MNIST/fake')
writer_real = SummaryWriter(f'GAN_MNIST/real')

step = 0

for epoch in range(num_epoch):
  for batch, (real, _) in enumerate(loader):
    real = real.view(-1,784).to(device)
    batch_size =  real.shape[0]

    # Train Discriminator -- > Loss --> Max E(D)log(D(real)) + E(G)log(1-D(G(fake)))
    optimizer_disc.zero_grad()

    noise = torch.randn((batch_size,z_dim)).to(device)
    fake = gen.forward(noise)

    disc_real = disc(real).view(-1)
    lossD_real = critrion(disc_real, torch.ones_like(disc_real))

    disc_fake = disc(fake).view(-1)
    lossD_fake = critrion(disc_fake, torch.zeros_like(disc_fake))

    lossD = (lossD_real + lossD_fake)/2

    lossD.backward(retain_graph = True)
    optimizer_disc.step()

    # Train Generator --> Min E(G)log(1-D(G(fake))) --- > Max (E(G)log(D(G(fake))))
    optimizer_gen.zero_grad()

    output = disc(fake).view(-1)
    lossG = critrion(output, torch.ones_like(output))
    lossG.backward()

    optimizer_gen.step()

    # Print Loss
    if batch == 0:
      print(f'Epoch {epoch}/{num_epoch}  Discriminator_Loss {lossD:.4f}  Generator_Loss {lossG:.4f} ')

    with torch.no_grad():
      fake = gen(fixed_noise).reshape(-1,1,28,28)
      data = real.reshape(-1,1,28,28)

      img_grid_fake = torchvision.utils.make_grid(fake,normalize=True)
      img_grid_real = torchvision.utils.make_grid(real,normalize=True)

      writer_fake.add_image("Mnist Fake Image ", img_grid_fake, global_step=step)
      writer_real.add_image("Mnist Real Image ", img_grid_real, global_step=step)

      step +=1
